In [69]:
import datetime
import pandas as pd 
import math as m
import statsmodels.formula.api as sm
import numpy as np
from data_process_final import gdp_annual, pop_annual, precip_annual, lk_lvl_annual, swe_annual


final = gdp_annual.merge(pop_annual, on = 'date', how = 'outer').merge(precip_annual, on = 'date', how = 'outer').merge(lk_lvl_annual, on = 'date', how = 'outer').merge(swe_annual, on = 'date', how = 'outer')
final['ln_gdp'] = np.log(final['gdp'])
final['ln_pop'] = np.log(final['population'])
final['ln_precip'] = np.log(final['precipitation'])
final['ln_lk_lvl'] = np.log(final['lake_level'])
final['ln_swe'] = np.log(final['swe'])
final['gdp_sq'] = np.square(final['gdp'])
final['ln_gdp_sq'] = np.log(final['gdp_sq'])
final['pop_100k'] = final['population']/100000
final['gdp_100k'] = final['gdp']/100000


In [72]:
final_test = final.drop(final.index[25:])
correlation = final_test.corr()
correlation

,gdp,population,precipitation,lake_level,swe,ln_gdp,ln_pop,ln_precip,ln_lk_lvl,ln_swe,gdp_sq,ln_gdp_sq,pop_100k,gdp_100k
gdp,1.000000,0.978253,-0.186993,-0.787347,-0.226238,0.982130,0.965556,-0.203054,-0.787431,-0.250635,0.984178,0.982130,0.978253,1.000000
population,0.978253,1.000000,-0.209226,-0.825911,-0.248787,0.993812,0.997911,-0.219344,-0.825975,-0.272521,0.930746,0.993812,1.000000,0.978253
precipitation,-0.186993,-0.209226,1.000000,-0.026200,0.418204,-0.205459,-0.214555,0.983481,-0.026233,0.394226,-0.166095,-0.205459,-0.209226,-0.186993
lake_level,-0.787347,-0.825911,-0.026200,1.000000,-0.063192,-0.831535,-0.834210,-0.035217,1.000000,-0.035250,-0.718938,-0.831535,-0.825911,-0.787347
swe,-0.226238,-0.248787,0.418204,-0.063192,1.000000,-0.236448,-0.254345,0.380074,-0.063169,0.983937,-0.216061,-0.236448,-0.248787,-0.226238
ln_gdp,0.982130,0.993812,-0.205459,-0.831535,-0.236448,1.000000,0.992874,-0.212228,-0.831592,-0.261111,0.934002,1.000000,0.993812,0.982130
ln_pop,0.965556,0.997911,-0.214555,-0.834210,-0.254345,0.992874,1.000000,-0.220868,-0.834263,-0.277985,0.908150,0.992874,0.997911,0.965556
ln_precip,-0.203054,-0.219344,0.983481,-0.035217,0.380074,-0.212228,-0.220868,1.000000,-0.035260,0.350604,-0.189536,-0.212228,-0.219344,-0.203054
ln_lk_lvl,-0.787431,-0.825975,-0.026233,1.000000,-0.063169,-0.831592,-0.834263,-0.035260,1.000000,-0.035243,-0.719041,-0.831592,-0.825975,-0.787431
ln_swe,-0.250635,-0.272521,0.394226,-0.035250,0.983937,-0.261111,-0.277985,0.350604,-0.035243,1.000000,-0.238464,-0.261111,-0.272521,-0.250635


In [49]:
first_reg = sm.ols('lake_level ~ swe ', data = final_test).fit()
first_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             lake_level   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.039
Method:                 Least Squares   F-statistic:                   0.09221
Date:                Wed, 19 Apr 2023   Prob (F-statistic):              0.764
Time:                        13:45:03   Log-Likelihood:                -64.285
No. Observations:                  25   AIC:                             132.6
Df Residuals:                      23   BIC:                             135.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4195.9951      1.951   2150.964      0.000    4191.960    4200.031
swe           -0.0107      0.035     -0.304      0.764      -0.084       0.062
==============================================================================
Omnibus:                        2.227   Durbin-Watson:                   0.211
Prob(Omnibus):                  0.328   Jarque-Bera (JB):                1.924
Skew:                           0.632   Prob(JB):                        0.382
Kurtosis:                       2.502   Cond. No.                         164.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [51]:
first_reg = sm.ols('lake_level ~ gdp', data = final_test).fit()
first_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             lake_level   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                     37.51
Date:                Wed, 19 Apr 2023   Prob (F-statistic):           3.01e-06
Time:                        13:48:51   Log-Likelihood:                -52.243
No. Observations:                  25   AIC:                             108.5
Df Residuals:                      23   BIC:                             110.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4202.0205      1.150   3655.345      0.000    4199.642    4204.399
gdp         -5.39e-05    8.8e-06     -6.125      0.000   -7.21e-05   -3.57e-05
==============================================================================
Omnibus:                        2.568   Durbin-Watson:                   0.684
Prob(Omnibus):                  0.277   Jarque-Bera (JB):                1.516
Skew:                           0.326   Prob(JB):                        0.469
Kurtosis:                       1.985   Cond. No.                     3.68e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.68e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [73]:
first_reg = sm.ols('lake_level ~ swe + precipitation + gdp_100k', data = final_test).fit()
first_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             lake_level   R-squared:                       0.689
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     15.49
Date:                Thu, 20 Apr 2023   Prob (F-statistic):           1.53e-05
Time:                        15:39:49   Log-Likelihood:                -49.747
No. Observations:                  25   AIC:                             107.5
Df Residuals:                      21   BIC:                             112.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      4205.8315      2.310   1820.851      0.000    4201.028    4210.635
swe              -0.0367      0.023     -1.593      0.126      -0.085       0.011
precipitation    -0.0962      0.136     -0.708      0.487      -0.379       0.187
gdp_100k         -5.8470      0.860     -6.797      0.000      -7.636      -4.058
==============================================================================
Omnibus:                        1.421   Durbin-Watson:                   0.518
Prob(Omnibus):                  0.491   Jarque-Bera (JB):                0.997
Skew:                           0.175   Prob(JB):                        0.607
Kurtosis:                       2.086   Cond. No.                         351.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
final_reg = sm.ols('ln_lk_lvl ~ ln_swe + ln_precip + ln_pop + ln_gdp_sq', data = final_test).fit()
final_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              ln_lk_lvl   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     19.38
Date:                Wed, 19 Apr 2023   Prob (F-statistic):           1.18e-06
Time:                        11:43:50   Log-Likelihood:                 164.02
No. Observations:                  25   AIC:                            -318.0
Df Residuals:                      20   BIC:                            -311.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.4235      0.048    175.622      0.000       8.323       8.524
ln_swe        -0.0005      0.000     -2.165      0.043      -0.001   -1.92e-05
ln_precip     -0.0005      0.000     -1.448      0.163      -0.001       0.000
ln_pop        -0.0055      0.005     -1.203      0.243      -0.015       0.004
ln_gdp_sq   9.929e-05      0.001      0.118      0.907      -0.002       0.002
==============================================================================
Omnibus:                        1.424   Durbin-Watson:                   0.766
Prob(Omnibus):                  0.491   Jarque-Bera (JB):                0.953
Skew:                           0.105   Prob(JB):                        0.621
Kurtosis:                       2.067   Cond. No.                     1.76e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.76e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""